In [1]:
import pickle
import pandas as pd
import glob2
import plotly.express as px
import plotly.graph_objects as go
from influxdb import InfluxDBClient
from plotly.subplots import make_subplots
import log_loader_time

Welcome to JupyROOT 6.16/00


Ricordati di generare i file di tempo (log_loader_time.py)

In [14]:
def extract_from_graal_and_save():
    dict_4_pd={
        "run":[],
        "subrun":[],
        "time":[],
        "Number_of_cluster_L1top_spX":[],
        "Number_of_cluster_L1bot_spX":[],
        "Number_of_cluster_L2top_spX":[],
        "Number_of_cluster_L2bot_spX":[],
        "Number_of_cluster_L1top_spV":[],
        "Number_of_cluster_L1bot_spV":[],
        "Number_of_cluster_L2top_spV":[],
        "Number_of_cluster_L2bot_spV":[],
        "Cluster_size_L1top_spX":[],
        "Cluster_size_L1bot_spX":[],
        "Cluster_size_L2top_spX":[],
        "Cluster_size_L2bot_spX":[],
        "Cluster_size_L1top_spV":[],
        "Cluster_size_L1bot_spV":[],
        "Cluster_size_L2top_spV":[],
        "Cluster_size_L2bot_spV":[],
        "Cluster_charge_L1top_spX":[],
        "Cluster_charge_L1bot_spX":[],
        "Cluster_charge_L2top_spX":[],
        "Cluster_charge_L2bot_spX":[],
        "Cluster_charge_L1top_spV":[],
        "Cluster_charge_L1bot_spV":[],
        "Cluster_charge_L2top_spV":[],
        "Cluster_charge_L2bot_spV":[]
    }
    time={}
    for filename, (run,)in glob2.iglob("/media/alb/Removibile/time_save/*", with_matches=True):
        if int(run)>395:
            with open (filename,"rb+") as savefile:
                start,end=pickle.load(savefile)
            time[int(run)]=start

            for filename, (subrun,)in glob2.iglob("/home/alb/GRAAL_out/srv_lab_mount/716392{}_*.txt".format(run), with_matches=True):
                with open (filename,'r') as graal_file:
                    if subrun in time[int(run)].keys():
                        dict_4_pd["run"].append(int(run))
                        dict_4_pd["subrun"].append(int(subrun))
                        dict_4_pd["time"].append(time[int(run)][subrun])
                        for line in graal_file.readlines():
                            for key in dict_4_pd:
                                if "X" in key:
                                    if key.split("_sp")[0].replace("_"," ") in line:
                                        dict_4_pd["{}_spX".format(key.split("_sp")[0])].append(float(line.split("-->")[1].split("|")[0].strip("\n").strip(" fC ")))
                                        dict_4_pd["{}_spV".format(key.split("_sp")[0])].append(float(line.split("-->")[2].strip("\n").strip(" fC ")))
                    else:
                        print ("Missing, R{}, S{}".format(run,subrun))
            print ("RUN {} done".format(run))
            cluster_pd=pd.DataFrame(dict_4_pd)
            cluster_pd=cluster_pd.sort_values(by=["run","subrun"])
            # cluster_pd['time'] = pd.to_datetime(cluster_pd['time'],unit='s')
            # cluster_pd=cluster_pd.set_index("time")
            # cluster_pd["hour"]=cluster_pd.index.hour
            # cluster_pd["day"]=cluster_pd.index.date
            savefilename="/media/alb/Removibile/graal_extract/cluster_dataframe"
            with open (savefilename,"wb+") as savefile:
                pickle.dump(cluster_pd,savefile)  
    return cluster_pd

In [15]:
cluster_pd=extract_from_graal_and_save()

RUN 396 done
RUN 397 done
RUN 400 done
RUN 405 done
RUN 406 done
RUN 409 done
RUN 410 done
RUN 411 done
RUN 412 done
RUN 418 done
RUN 419 done
RUN 420 done


In [16]:
cluster_pd['time'] = pd.to_datetime(cluster_pd['time'],unit='s')
cluster_pd=cluster_pd.set_index("time")
cluster_pd["hour"]=cluster_pd.index.hour
cluster_pd["day"]=cluster_pd.index.date

In [17]:
cluster_pd_1h=cluster_pd.resample("1h").mean()